In [1]:
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from analysis_twolayer import read_n, classify_imgs2

['/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py', '-f', 'true', 'true', 'true', 'true', 'true', 'model_runs/-f/classify_log.csv', 'model_runs/-f/classifymodel_0.ckpt', 'model_runs/-f/classifymodel_', 'model_runs/-f/zzzdraw_data_5000.npy', 'false', 'true', 'false', 'false', 'true']
Instructions for updating:
Use `argmax` instead
Instructions for updating:
Use `argmax` instead
analysis_twolayer.py


In [2]:
model_name = "DRAM_twolayer_filterimg_norm_run_3"
iter_list = [0, 250, 1000, 4000, 16000, 32000, 64000, 125000, 250000, 500000, 1000000, 1500000, 2000000, 2300000]#, 5000000]
max_blobs = 15
min_blobs = 1
num_runs = 1
batch_size = 100
num_iters = len(iter_list)
output_size = max_blobs - min_blobs + 1 # 15
data_directory = "data/" + model_name + "/"
m = 0.5

confidence_all_runs = np.zeros([num_runs, num_iters, output_size, output_size]) 
choice_all_runs = np.zeros([num_runs, num_iters, output_size, output_size]) 

In [3]:
def fill_matrix(path, iteration):
    """Fill the confidence and choice matrices for one run at one iteration."""
    
    data = None
    num_imgs = batch_size
    imgs_data = classify_imgs2(iteration, True, num_imgs, path=path) # new_imgs = True
        
    confidence_one_run = np.zeros([output_size, output_size])
    choice_one_run = np.zeros([output_size, output_size])

    for nb in range(output_size): 
        num_blobs = nb + min_blobs # 1 to 15
        confidence_hist = np.zeros(output_size)
        choice_hist = np.zeros(output_size)
        num_imgs_with_num_blobs = 0.00001

        for idx, data in enumerate(imgs_data):

            if data["label"][nb] == 1: # data is for an image with num_blobs blobs
                num_imgs_with_num_blobs += 1

                # Histogram of softmaxes
                confidence_hist += data["classifications"][0]

                # Histogram of choices
                choice = np.argmax(data["classifications"][0])
                choice_list = [0] * output_size
                choice_list[choice] = 1
                choice_hist += choice_list

        confidence_hist = confidence_hist / num_imgs_with_num_blobs
        confidence_one_run[nb] = confidence_hist.tolist()

        choice_hist = choice_hist / num_imgs_with_num_blobs
        choice_one_run[nb] = choice_hist.tolist()
        
#     print("Confidence (One Run): ",confidence_one_run[nb])
#     print(confidence_one_run.tolist())
#     print("Choice (One Run): ",choice_one_run[nb])
#     print(choice_one_run.tolist())
    return confidence_one_run, choice_one_run

In [4]:
for run in range(num_runs):
    path = 'model_runs/' + model_name # '/run_' if all the runs are stored in one folder
    for i, iteration in enumerate(iter_list):
        confidence_all_runs[run, i], choice_all_runs[run, i] = fill_matrix(path, iteration)

INFO:tensorflow:Restoring parameters from model_runs/DRAM_twolayer_filterimg_norm_run_2/classifymodel_0.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_twolayer_filterimg_norm_run_2/classifymodel_250.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_twolayer_filterimg_norm_run_2/classifymodel_1000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_twolayer_filterimg_norm_run_2/classifymodel_4000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_twolayer_filterimg_norm_run_2/classifymodel_16000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_twolayer_filterimg_norm_run_2/classifymodel_32000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_twolayer_filterimg_norm_run_2/classifymodel_64000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_twolayer_filterimg_norm_run_2/classifymodel_125000.ckpt
INFO:tensorflow:Restoring parameters from model_runs/DRAM_twolayer_filterimg_norm_run_2/classifymodel_2

In [5]:
def adj_all_runs(all_runs_matrix):
    """Adjust all the matrices so tick marks start with 1."""
    
    new_all_runs_matrix = np.zeros([num_runs, num_iters, output_size + 1, output_size + 1])
    for m, matrix in enumerate(all_runs_matrix):
        for i in range(num_iters):
            new_all_runs_matrix[m, i] = adj_matrix(matrix[i])
    return new_all_runs_matrix


def adj_matrix(matrix):
    """Adjust the matrix so tick marks start with 1."""
    
    a = np.zeros([1, output_size])
    temp = np.vstack((a, matrix))
    b = np.zeros([output_size + 1, 1])
    new_matrix = np.hstack((b, temp))
    return new_matrix


confidence_all_runs_adj = adj_all_runs(confidence_all_runs)
choice_all_runs_adj = adj_all_runs(choice_all_runs)

In [6]:
confidence_avg = np.mean(confidence_all_runs_adj, axis=0)
choice_avg = np.mean(choice_all_runs_adj, axis=0)

In [7]:
def plot_confidence(iter_idx, it, run=None):
    """Plot the confidence heatmap."""
    
    if run is None:
        matrix = confidence_avg[iter_idx]
        plot_title = "Confidence after %d Iterations" % (it)

    else:
        matrix = confidence_all_runs_adj[run, iter_idx]
        plot_title = "Confidence at Run %d after %d Iterations" % (run + 1, it)
    plot_heatmap(matrix, plot_title)


def plot_choice(iter_idx, it, run=None):
    """Plot the choice heatmap."""
    
    if run is None:
        matrix = choice_avg[iter_idx]
        plot_title = "Choice after %d Iterations" % (it)
    else:
        matrix = choice_all_runs_adj[run, iter_idx]
        plot_title = "Choice at Run %d after %d Iterations" % (run + 1, it)
    plot_heatmap(matrix, plot_title)
    
    
def plot_heatmap(matrix, plot_title):
    """Plot heatmap."""
    
    data = [go.Heatmap(
        z=matrix,
        colorscale="Jet"
    )]

    layout = go.Layout(
        title=plot_title,
        yaxis=dict(
#             range=[max_blobs + m, min_blobs - m],
            range=[9 + m, min_blobs - m],
            title="True Class",
            dtick=1,
            tickcolor='#FFF'
        ),
        xaxis=dict(
            range=[min_blobs - m, max_blobs + m],
            title="Predicted Class",
            dtick=1,
            tickcolor='#FFF'
        ),
        width=500,
        height=500,
        plot_bgcolor="#000",
        paper_bgcolor="#000",
        font=dict(
            color="#FFF"
        ),
        titlefont=dict(
            color="#FFF"
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [8]:
for i, it in enumerate(iter_list):
    #plot_confidence(i, it)
    plot_choice(i, it)